# 0. 환경설정

In [66]:
import os
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
os.chdir(path = 'code')

In [73]:
os.listdir()

['.DS_Store',
 '.gitignore',
 '.ipynb_checkpoints',
 '00_Library_Installation.ipynb',
 '22시각화-Copy1.ipynb',
 '22시각화.ipynb',
 '22형태소분석-Copy1.ipynb',
 '22형태소분석.ipynb',
 'A1_Naver_News_Link_Func.ipynb',
 'A2_Naver_News_Body_Func.ipynb',
 'A3_Naver_News_Reply_Func.ipynb',
 'B1_Text_Mining.ipynb',
 'B2_Visualization.ipynb',
 'B3_Topic_Modeling.ipynb',
 'data',
 'df22.pkl',
 'df23.pkl',
 'venv']

In [74]:
df23 = pd.read_csv('data/2023_log.csv')
df24 = pd.read_csv('data/2024_log.csv')

In [75]:
df23.head()

,Press,Headline,Analysis
0,연합뉴스,"'내년 첫 전기차 출시' 中샤오미 ""테슬라 따라잡을 준비됐다""",긍정
1,YTN,"'자국 우선주의' 허들 만난 '전기차', 묘수는?",중립
2,연합뉴스,"애플 따라하던 中 샤오미, 전기차는 먼저 치고나가",긍정
3,연합뉴스,"美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려",부정
4,매일경제,주춤한 전기차 … 내년 보급형 몰려온다,중립


In [5]:
df23['Headline'] = df23['Headline'].str.replace(pat = '<.+?>', repl = '', regex = True)

In [6]:
df23['Headline'].head()

0    '내년 첫 전기차 출시' 中샤오미 "테슬라 따라잡을 준비됐다"
1           '자국 우선주의' 허들 만난 '전기차', 묘수는?
2           애플 따라하던 中 샤오미, 전기차는 먼저 치고나가
3      美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려
4                 주춤한 전기차 … 내년 보급형 몰려온다
Name: Headline, dtype: object

In [7]:
os.getcwd()

'C:\\Users\\kim jeongah\\OneDrive\\Desktop\\DMF\\DBR_Project\\DBR_Project\\code'

In [8]:
pd.to_pickle(obj = df23, filepath_or_buffer = 'df23.pkl')

In [9]:
os.listdir()

['.DS_Store',
 '.gitignore',
 '.ipynb_checkpoints',
 '00_Library_Installation.ipynb',
 '22시각화-Copy1.ipynb',
 '22시각화.ipynb',
 '22형태소분석-Copy1.ipynb',
 '22형태소분석.ipynb',
 'A1_Naver_News_Link_Func.ipynb',
 'A2_Naver_News_Body_Func.ipynb',
 'A3_Naver_News_Reply_Func.ipynb',
 'B1_Text_Mining.ipynb',
 'B2_Visualization.ipynb',
 'B3_Topic_Modeling.ipynb',
 'data',
 'df22.pkl',
 'df23.pkl',
 'venv']

In [10]:
os.chdir(path = '../data')

In [11]:
sorted(os.listdir())

['22_Prep.pkl',
 '23_Prep.pkl',
 'Naver_News_List.pkl',
 'Naver_News_Reply.pkl',
 'Text_Prep.pkl',
 'df22.pkl',
 'df23.pkl']

In [12]:
df23 = pd.read_pickle(filepath_or_buffer = 'df23.pkl')

In [13]:
df23.head()

,Press,Headline,Analysis
0,연합뉴스,"'내년 첫 전기차 출시' 中샤오미 ""테슬라 따라잡을 준비됐다""",긍정
1,YTN,"'자국 우선주의' 허들 만난 '전기차', 묘수는?",중립
2,연합뉴스,"애플 따라하던 中 샤오미, 전기차는 먼저 치고나가",긍정
3,연합뉴스,"美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려",부정
4,매일경제,주춤한 전기차 … 내년 보급형 몰려온다,중립


In [14]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Press     781 non-null    object
 1   Headline  781 non-null    object
 2   Analysis  781 non-null    object
dtypes: object(3)
memory usage: 18.4+ KB


In [15]:
sens = df23['Headline']

In [16]:
sens.head(100)

0         '내년 첫 전기차 출시' 中샤오미 "테슬라 따라잡을 준비됐다"
1                '자국 우선주의' 허들 만난 '전기차', 묘수는?
2                애플 따라하던 中 샤오미, 전기차는 먼저 치고나가
3           美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려
4                      주춤한 전기차 … 내년 보급형 몰려온다
                       ...                  
95      "순식간에 1,500℃까지 발화"...전기차 화재 진압 방법 고심
96                전기차 사러 왔다가 하이브리드 몰고 나가는 이유
97                        제동 걸린 전기차 전환[오후여담]
98        캐나다, 2035년까지 모든 차량 ‘전기차 의무화’ 공식 발표
99    '프리미엄' 볼보도 4000만원대 전기차 출시…"높은 가격이 걸림돌"
Name: Headline, Length: 100, dtype: object

In [17]:
i=0
sens.iloc[1]

"'자국 우선주의' 허들 만난 '전기차', 묘수는?"

# 0.1 빈도수 정확도를 높이기 위한 튜닝

In [18]:
# sens에서 지정한 패턴이 아닌 글자를 공백으로 변경하고 sens에 재할당
sens = sens.str.replace(pat = '[^가-힣A-Za-z0-9.]|&.+;', repl = '', regex = True)

# 일부 단어를 변경함
sens = sens.str.replace(pat = 'e트론', repl = '아우디', regex = True)
sens = sens.str.replace(pat = '中', repl = '중국', regex = True)
sens = sens.str.replace(pat = '美', repl = '미국', regex = True)
sens = sens.str.replace(pat = '獨', repl = '독일', regex = True)
# sens = sens.str.replace(pat = '전기', repl = '전기차', regex = True)
sens = sens.str.replace(pat = '전기', repl = '', regex = True)
sens = sens.str.replace(pat = '기차', repl = '', regex = True)
sens = sens.str.replace(pat = '차다', repl = '', regex = True)

# 1. 형태소 분석

In [19]:
kiwistop = Stopwords()

In [20]:
kiwi = Kiwi()

In [21]:
sens[i]

'내년첫차출시샤오미테슬라따라잡을준비됐다'

In [22]:
kiwi.tokenize(text = sens[i], stopwords = kiwistop)

[Token(form='내년', tag='NNG', start=0, len=2),
 Token(form='첫차', tag='NNG', start=2, len=2),
 Token(form='출시', tag='NNG', start=4, len=2),
 Token(form='샤오미', tag='NNP', start=6, len=3),
 Token(form='테슬라', tag='NNP', start=9, len=3),
 Token(form='따라잡', tag='VV-R', start=12, len=3),
 Token(form='준비', tag='NNG', start=16, len=2)]

In [23]:
words = ['인도', 'e트론']

In [24]:
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNP', score = 15)

In [25]:
tokens = kiwi.tokenize(text = sens[i], stopwords = kiwistop)

tokens

[Token(form='내년', tag='NNG', start=0, len=2),
 Token(form='첫차', tag='NNG', start=2, len=2),
 Token(form='출시', tag='NNG', start=4, len=2),
 Token(form='샤오미', tag='NNP', start=6, len=3),
 Token(form='테슬라', tag='NNP', start=9, len=3),
 Token(form='따라잡', tag='VV-R', start=12, len=3),
 Token(form='준비', tag='NNG', start=16, len=2)]

In [26]:
print(f'형태:{tokens[5].form}')
print(f'품사:{tokens[5].tag}')

형태:따라잡
품사:VV-R


In [27]:
pos1, pos2 = ['VV', 'VA'], ['NNG', 'NNP']

In [28]:
# 형태소의 품사가 용언과 체언인 형태소만 선택하고 품사가 용언인 형태소에 
# 종결어미 '다'를 결합하여 리스트로 반환

tokens = [token.form + '다' if token.tag in pos1 else token.form 
          for token in tokens if token.tag in pos1 + pos2]

tokens

['내년', '첫차', '출시', '샤오미', '테슬라', '준비']

# 2. 말뭉치 생성

In [29]:
# 말뭉치를 저장할 빈 리스트 생성
corpus = list()

# sens의 원소(문서)를 형태소 분석하고 일부 품사를 선택하여 말뭉치에 추가
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = kiwistop)
    tokens = [token.form + '다' if token.tag in pos1 else token.form 
              for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [30]:
# 형태소 분석 결과(corpus)의 처음 5개 원소 확인
# [참고] 원본 문서와 결과를 비교하여 사용자 사전에 추가하는 것을 반복해야 함
for i in range(5):
    print(corpus[i])

['내년', '첫차', '출시', '샤오미', '테슬라', '준비']
['자국', '우선주의', '허들', '만나다', '차', '묘수']
['애플', '샤오미', '차', '치다']
['차', '전환', '정부', '지원', '수요', '감소', '기대', '느리다']
['차', '내년', '보급', '몰려오다']


In [31]:
for i in range(5):
    print(df23['Headline'].iloc[i], '\n')

'내년 첫 전기차 출시' 中샤오미 "테슬라 따라잡을 준비됐다" 

'자국 우선주의' 허들 만난 '전기차', 묘수는? 

애플 따라하던 中 샤오미, 전기차는 먼저 치고나가 

美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려 

주춤한 전기차 … 내년 보급형 몰려온다 



# 3. 문서-단어 행렬 생성

In [32]:
# corpus의 각 원소(리스트)를 하나의 문자열로 결합하여 corpus에 재할당
# [참고] 문서-단어 행렬 생성하려면 corpus 원소가 문자열이어야 함
corpus = [' '.join(i) for i in corpus]

# corpus의 처음 5개 확인
corpus[0:5]

['내년 첫차 출시 샤오미 테슬라 준비',
 '자국 우선주의 허들 만나다 차 묘수',
 '애플 샤오미 차 치다',
 '차 전환 정부 지원 수요 감소 기대 느리다',
 '차 내년 보급 몰려오다']

In [33]:
# 문서별 단어의 tf를 계산하는 객체 생성
# [참고] TfidfVectorizer()는 단어의 tf-idf를 반환
cv = CountVectorizer()

In [34]:
# 단어의 tf를 성분으로 갖는 문서-단어 행렬 생성
dtm = cv.fit_transform(raw_documents = corpus).toarray()

# dtm 확인
# [참고] dtm은 2차원 행렬임
dtm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [35]:
# cv 객체에 저장된 단어 목록 확인
# [참고] 알파벳이 모두 소문자로 바뀌었음
cv.get_feature_names_out()

array(['1위비야디', '3사', '3연타', ..., '흔들리다', '희토', '히터'], dtype=object)

In [36]:
# dtm을 데이터프레임으로 변환
dtm = pd.DataFrame(data = dtm, columns = cv.get_feature_names_out())

In [37]:
# dtm의 열이름에서 알파벳을 대문자로 변경
dtm.columns = dtm.columns.str.upper()

In [38]:
dtm.head()

,1위비야디,3사,3연타,KG르노차,LG이노텍,LG전자,LS머트,M피소드차,W차,가격,...,후발,후진,훈풍,휘발유,휘청이다,휴대폰,흑자,흔들리다,희토,히터
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
dtm.shape

(781, 1599)

In [40]:
tfs = dtm.sum().sort_values(ascending = False)

In [41]:
# tfs의 상위 10개 및 하위 10개 확인
display(tfs.head(n = 10))
display(tfs.tail(n = 10))

배터리    101
보조금     94
시장      56
테슬라     54
충전      52
판매      39
중국      38
공장      37
가격      34
생산      33
dtype: int64

석준     1
서부     1
상하이    1
서류     1
서도     1
생활     1
생존법    1
생산량    1
상황     1
히터     1
dtype: int64

In [42]:
# tfs가 20 이하인 인덱스(단어)를 dtm에서 제거
# [참고] 단어 빈도수를 높일수록 dtm의 열(차원) 개수가 감소함
threshold = 20
dtm = dtm.drop(columns = tfs.loc[tfs.le(threshold)].index)

In [43]:
# dtm의 행 개수와 열 개수 확인
# [참고] 단어 빈도수가 매우 작은 일부 단어를 삭제함으로써 차원(열) 축소 가능
# [참고] dtm의 차원을 축소하면 행렬곱 연산을 빠르게 실행할 수 있음
dtm.shape

(781, 22)

In [44]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\kim jeongah\\OneDrive\\Desktop\\DMF\\DBR_Project\\DBR_Project\\data'

In [45]:
# corpus, tfs 및 dtm을 하나의 pkl 파일로 저장
pd.to_pickle(obj = [corpus, tfs, dtm], filepath_or_buffer = '23_Prep.pkl')

In [46]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['22_Prep.pkl',
 '23_Prep.pkl',
 'df22.pkl',
 'df23.pkl',
 'Naver_News_List.pkl',
 'Naver_News_Reply.pkl',
 'Text_Prep.pkl']

In [47]:
df23['Analysis'].value_counts(normalize=True)

Analysis
부정    0.386684
중립    0.320102
긍정    0.293214
Name: proportion, dtype: float64

In [76]:
df24['Analysis'].value_counts(normalize=True)

Analysis
중립    0.358621
긍정    0.353103
부정    0.288276
Name: proportion, dtype: float64

In [78]:
csv_files = [
]

dfs = [
    '/data/2020_log.csv',
    '/data/2021_log.csv',
    '/data/2022_log.csv',
    '/data/2023_log.csv',
    '/data/2024_log.csv',
]

for i, file in enumerate(csv_files):
    df = pd.read_csv(file)
    dfs.append(df)

merged_df = pd.concat(dfs, axis=0, ignore_index=True)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid